In [18]:
# This reload library is just used for developing the REPUBLIC hOCR parser 
# and can be removed once this module is stable.
%reload_ext autoreload
%autoreload 2


# This is needed to add the repo dir to the path so jupyter
# can load the republic modules directly from the notebooks
import os
import sys
repo_name = 'republic-project'
repo_dir = os.path.split(os.getcwd())[0].split(repo_name)[0] + repo_name
print("adding project dir to path:", repo_dir)
if repo_dir not in sys.path:
    sys.path.append(repo_dir)



adding project dir to path: /Users/marijnkoolen/Code/Huygens/republic-project


In [19]:
from republic.elastic.republic_elasticsearch import initialize_es
from republic.config.republic_config import set_config_inventory_num

inv_num = 3820
inv_config = set_config_inventory_num(inv_num, ocr_type="pagexml")

rep_es = initialize_es(host_type="external")

rep_es

In [2]:
import pandas as pd
import numpy as np

ind_file_early = '../../data/indices/index_entries-3763-3804-latest.csv.gz'
ind_file_late = '../../data/indices/index_entries-3806-3864-latest.csv.gz'


In [7]:
df_early = pd.read_csv(ind_file_early, compression='gzip', sep='\t')
df_late = pd.read_csv(ind_file_late, compression='gzip', sep='\t')

df_early['inventory_num'] = df_early.first_line_id.apply(lambda x: int(x.split('_')[2]))


In [46]:
import re

def clean_main_term(main_term):
    main_term = re.sub(r"^\W+","", main_term)
    main_term = re.sub(r"\W+$","", main_term)
    return main_term

df_late['main_term'] = df_late.main_term.apply(clean_main_term)

In [5]:
df_early.head(2)

,lemma,main_term,text,first_line_id,first_line_scan_id,first_line_page_id,first_line_column_id,last_line_id,last_line_scan_id,last_line_page_id,last_line_column_id
0,nee Winter quartieren,nee,"nee Winter quartieren, 918.",NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...
1,nee Winter quartieren,nee,aerte tifthop van Gran Brief van felicitatie o...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...,NL-HaNA_1.01.02_3763_0626,NL-HaNA_1.01.02_3763_0626-page-1251,NL-HaNA_1.01.02_3763_0626-column-2471-1318-894...


In [104]:
print(df_late.shape)
df_late.head()

(225182, 9)


,main_term,sub_lemma,date_locator,date_locator_day,date_locator_month,page_locator,inventory_num,scan_id,page_id
0,Aalbregt,VX Fygeslagen sijn Rey VW versoek om WES uit d...,16 July,16,Jul,419,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
1,Aardenburg,Sie op Vlaanderen.,NaN,NaN,NaN,NaN,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
2,Aarle-Rixtel,"Sie op 's Hertogenbosch en Meyerye,.",NaN,NaN,NaN,NaN,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
3,d' AblaingvanGies,Requeste van de Heer d'Ahlaing van Gies,NaN,NaN,NaN,NaN,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007
4,Jenburg,"senburg, verso-kende dat het Jagtgerigt van Vl...",4 Mey,4,Mey,266,3806,NL-HaNA_1.01.02_3806_0007,NL-HaNA_1.01.02_3806_0007


In [47]:
df_early[['main_term', 'inventory_num']].drop_duplicates().main_term.value_counts()

Brieven         41
Bergen          40
Admiraliteyt    40
Breda           40
Deurs           39
                ..
Nieuhuys         1
Nootstock        1
Okctroyen        1
Onkruyt          1
Zwitzerse        1
Name: main_term, Length: 12484, dtype: int64

In [48]:
df_late[['main_term', 'inventory_num']].drop_duplicates().main_term.value_counts()

Zeeland              47
Gelderland           47
Holland              47
Vriesland            47
Marocco              46
                     ..
gemeeneluisdeien      1
WViddelen             1
gemeene               1
Michiels,cum suis     1
ten,Zyi hoff          1
Name: main_term, Length: 31783, dtype: int64

In [103]:
df_full = pd.concat([df_early[['main_term', 'inventory_num']], df_late[['main_term', 'inventory_num']]])

print('number of main terms:', df_full.main_term.nunique())
print('number of sublemmas:', df_full.shape[0])



number of main terms: 42110
number of sublemmas: 365495


In [86]:
s_inv = df_full.drop_duplicates().main_term.value_counts()

s_inv[s_inv > 10]



Utrecht      77
Pruyssen     75
Pasporten    72
Consenten    71
Holland      71
             ..
Lynslager    11
Heeuft       11
XV           11
Eersel       11
Fenema       11
Name: main_term, Length: 998, dtype: int64

In [88]:
s_res = df_full.main_term.value_counts()

s_res[s_res > 10]



Pasporten                  5809
Militaire                  4877
Holland                    3831
Raad                       3789
Vrankryk                   3694
                           ... 
Frederiks                    11
Buytensandtsche              11
Schoolen,Schoolmeesters      11
MNMavigate                   11
Confesteurs                  11
Name: main_term, Length: 4840, dtype: int64

In [100]:
g_res = s_res.reset_index(name="# Sublemma's").rename(columns={'index': 'Hoofdterm'})
g_inv = s_inv.reset_index(name="# Indices").rename(columns={'index': 'Hoofdterm'})
g = pd.merge(g_inv, g_res, on="Hoofdterm")

g.head(20)
g.sort_values("# Sublemma's", ascending=False).head(20)

,Hoofdterm,# Indices,# Sublemma's
2,Pasporten,72,5809
35,Militaire,45,4877
4,Holland,71,3831
256,Raad,25,3789
47,Vrankryk,43,3694
67,Admiraliteyt,40,3526
52,Jaaken,42,3081
61,Hertogenbosch,41,3048
1,Pruyssen,75,2861
6,Zeeland,66,2376


In [108]:
aggs = {
    'invs': {
        'terms': {'field': 'metadata.inventory_num', 'size': 1000}
    }
}

response = rep_es.es_anno.search(index='pages', aggs=aggs, size=0)
buckets = response['aggregations']['invs']['buckets']
sorted([b['key'] for b in buckets])
sorted(buckets, key = lambda x: x['key'])

[{'key': 3281, 'doc_count': 830},
 {'key': 3282, 'doc_count': 1166},
 {'key': 3283, 'doc_count': 1697},
 {'key': 3284, 'doc_count': 1079},
 {'key': 3285, 'doc_count': 2118},
 {'key': 3286, 'doc_count': 1723},
 {'key': 3287, 'doc_count': 1544},
 {'key': 3288, 'doc_count': 1553},
 {'key': 3289, 'doc_count': 1368},
 {'key': 3760, 'doc_count': 1504},
 {'key': 3761, 'doc_count': 1477},
 {'key': 3762, 'doc_count': 1352},
 {'key': 3763, 'doc_count': 1286},
 {'key': 3764, 'doc_count': 1550},
 {'key': 3765, 'doc_count': 1473},
 {'key': 3766, 'doc_count': 1595},
 {'key': 3767, 'doc_count': 1528},
 {'key': 3768, 'doc_count': 1699},
 {'key': 3769, 'doc_count': 1341},
 {'key': 3770, 'doc_count': 1236},
 {'key': 3771, 'doc_count': 1146},
 {'key': 3772, 'doc_count': 1178},
 {'key': 3773, 'doc_count': 1207},
 {'key': 3774, 'doc_count': 1057},
 {'key': 3775, 'doc_count': 1027},
 {'key': 3776, 'doc_count': 1003},
 {'key': 3777, 'doc_count': 948},
 {'key': 3778, 'doc_count': 925},
 {'key': 3779, 'doc_cou

In [50]:
aggs = {
    'invs': {
        'terms': {'field': 'metadata.inventory_num', 'size': 1000}
    }
}

aggs = {
    'types': {
        'terms': {'field': 'columns.text_regions.type.keyword', 'size': 1000}
    }
}

response = rep_es.es_anno.search(index='pages', aggs=aggs, size=0)
buckets = response['aggregations']['types']['buckets']
sorted([b['key'] for b in buckets])
sorted(buckets, key = lambda x: x['key'])

[{'key': 'attendance', 'doc_count': 1684},
 {'key': 'column', 'doc_count': 10006},
 {'key': 'header', 'doc_count': 7214},
 {'key': 'heading', 'doc_count': 1},
 {'key': 'index', 'doc_count': 8943},
 {'key': 'index_column', 'doc_count': 435},
 {'key': 'main', 'doc_count': 24469},
 {'key': 'marginalia', 'doc_count': 8812},
 {'key': 'pagexml_doc', 'doc_count': 24575},
 {'key': 'resolution', 'doc_count': 21692},
 {'key': 'text_region', 'doc_count': 24575}]

In [72]:
from collections import Counter

from elasticsearch import Elasticsearch
from elasticsearch.exceptions import ElasticsearchException


def scroll_hits(es: Elasticsearch, query: dict, index: str,
                size: int = 100, scroll: str = '2m', in_bulk: bool = False) -> iter:
    if query is None or len(query.keys()) == 0:
        response = es.search(index=index, scroll=scroll, size=size)
    else:
        response = es.search(index=index, scroll=scroll, size=size, query=query)
    sid = response['_scroll_id']
    scroll_size = response['hits']['total']
    print('total hits:', scroll_size, "\thits per scroll:", len(response['hits']['hits']))
    if type(scroll_size) == dict:
        scroll_size = scroll_size['value']
    # Start scrolling
    while scroll_size > 0:
        if in_bulk:
            yield response['hits']['hits']
        else:
            for hit in response['hits']['hits']:
                yield hit
        try:
            response = es.scroll(scroll_id=sid, scroll=scroll)
        except ElasticsearchException:
            print("retrieval failed for query:")
            print(query)
            print("last successful hit:", response["hits"]["hits"][0]["_id"])
            raise
        # Update the scroll ID
        sid = response['_scroll_id']
        # Get the number of results that we returned in the last scroll
        scroll_size = len(response['hits']['hits'])
        # Do something with the obtained page
    # remove scroll context
    try:
        es.clear_scroll(scroll_id=sid)
    except ElasticsearchException:
        print('WARNING: no scroll id found when clearing scroll at end of scroll with query:')
        print(query)
        pass
    
    
def get_marginalia(page):
    trs = [tr for col in page['columns'] for tr in col['text_regions']]
    marginalia_trs = [tr for tr in trs if 'marginalia' in tr['type'] and 'lines' in tr]
    return [' '.join([line['text'] for line in tr['lines'] if line['text']]) for tr in marginalia_trs]
    

query = {
    'bool': {
        'must': [
            {'match': {'columns.text_regions.type': 'marginalia'}}
        ]
    }
}

marg_count = Counter()

rep_es.es_anno.search(index='pages', query=query, size=0, track_total_hits=True)
for hi, hit in enumerate(scroll_hits(rep_es.es_anno, query, 'pages', size = 2)):
    page = hit['_source']
    marginalia = get_marginalia(page)
    marg_count.update(marginalia)
    if (hi+1) % 100 == 0:
        print(hi+1, 'pages parsed')
    

total hits: {'value': 8812, 'relation': 'eq'} 	hits per scroll: 2
10 pages parsed
20 pages parsed
30 pages parsed
40 pages parsed
50 pages parsed
60 pages parsed
70 pages parsed
80 pages parsed
90 pages parsed
100 pages parsed
110 pages parsed
120 pages parsed
130 pages parsed
140 pages parsed
150 pages parsed
160 pages parsed
170 pages parsed
180 pages parsed
190 pages parsed
200 pages parsed
210 pages parsed
220 pages parsed
230 pages parsed
240 pages parsed
250 pages parsed
260 pages parsed
270 pages parsed
280 pages parsed
290 pages parsed
300 pages parsed
310 pages parsed
320 pages parsed
330 pages parsed
340 pages parsed
350 pages parsed
360 pages parsed
370 pages parsed
380 pages parsed
390 pages parsed
400 pages parsed
410 pages parsed
420 pages parsed
430 pages parsed
440 pages parsed
450 pages parsed
460 pages parsed
470 pages parsed
480 pages parsed
490 pages parsed
500 pages parsed
510 pages parsed
520 pages parsed
530 pages parsed
540 pages parsed
550 pages parsed
560 page

4590 pages parsed
4600 pages parsed
4610 pages parsed
4620 pages parsed
4630 pages parsed
4640 pages parsed
4650 pages parsed
4660 pages parsed
4670 pages parsed
4680 pages parsed
4690 pages parsed
4700 pages parsed
4710 pages parsed
4720 pages parsed
4730 pages parsed
4740 pages parsed
4750 pages parsed
4760 pages parsed
4770 pages parsed
4780 pages parsed
4790 pages parsed
4800 pages parsed
4810 pages parsed
4820 pages parsed
4830 pages parsed
4840 pages parsed
4850 pages parsed
4860 pages parsed
4870 pages parsed
4880 pages parsed
4890 pages parsed
4900 pages parsed
4910 pages parsed
4920 pages parsed
4930 pages parsed
4940 pages parsed
4950 pages parsed
4960 pages parsed
4970 pages parsed
4980 pages parsed
4990 pages parsed
5000 pages parsed
5010 pages parsed
5020 pages parsed
5030 pages parsed
5040 pages parsed
5050 pages parsed
5060 pages parsed
5070 pages parsed
5080 pages parsed
5090 pages parsed
5100 pages parsed
5110 pages parsed
5120 pages parsed
5130 pages parsed
5140 pages

In [106]:
sum(marg_count.values())

17195

In [73]:
for marg, count in marg_count.most_common(100):
    print(f"{marg: <80}{count: >6}")

                                                                                  3401
Sasburch                                                                            77
Brasser                                                                             63
Brunincx                                                                            47
Pels                                                                                46
Amerongen                                                                           41
Werckendam                                                                          40
Idem                                                                                37
Malapert                                                                            32
Boreel                                                                              26
Pels ...                                                                            22
Haren                                      

In [77]:
clean_marg_count = Counter()
for marg, count in marg_count.most_common():
    clean_marg = re.sub(r"\W+$", '', marg)
    clean_marg = re.sub(r"^\W+", '', clean_marg)
    clean_marg_count[clean_marg] += count
    
    
for marg, count in clean_marg_count.most_common(100):
    print(f"{marg: <80}{count: >6}")

                                                                                  3401
Sasburch                                                                           191
Pels                                                                               102
Malapert                                                                            81
Amerongen                                                                           78
Brasser                                                                             69
Brunincx                                                                            57
Werckendam                                                                          51
Boreel                                                                              47
Idem                                                                                45
Vrijbergen                                                                          32
Haren                                      

In [83]:
data = [{'main_term': marg, 'count': count} for marg, count in clean_marg_count.most_common()]

handwritten = pd.DataFrame(data=data)

handwritten[handwritten['count'] > 10]

,main_term,count
0,,3401
1,Sasburch,191
2,Pels,102
3,Malapert,81
4,Amerongen,78
5,Brasser,69
6,Brunincx,57
7,Werckendam,51
8,Boreel,47
9,Idem,45


In [105]:
225182 + 140313

365495